In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
import datetime
register_matplotlib_converters()

In [2]:

with open('Data/Dodgers.data','rb') as f:
    traffic = pd.read_csv(f,names=('Datetime','count'),encoding='ISO 8859-1')
    traffic['Datetime'] = pd.to_datetime(traffic.Datetime,format= '%m/%d/%Y %H:%M')
    

with open('Data/Dodgers.events','rb') as f:
    events = pd.read_csv(f,names=('Date','end_event','start_event','Game attendance','Away team','W/L score'),encoding='ISO 8859-1')
    events['start_event'] = pd.to_datetime(events.Date+'_'+events.start_event,format= '%m/%d/%y_%H:%M:%S')
    events['end_event'] = pd.to_datetime(events.Date+'_'+events.end_event,format= '%m/%d/%y_%H:%M:%S')
    events['Date'] = pd.to_datetime(events.Date,format= '%m/%d/%y')
    
with open('Data/CalIt2.data','r') as f:
    calTraffic = pd.read_csv(f,names=('direction','date','time','num'),encoding='iso-8859-1')
    calTraffic['datetime'] = pd.to_datetime(calTraffic.date+'_'+calTraffic.time,format= '%m/%d/%y_%H:%M:%S')
    calTraffic['date'] = pd.to_datetime(calTraffic.date,format= '%m/%d/%y')
    
with open('Data/CalIt2.events','rb') as f:
    calEvent= pd.read_csv(f,names=('Date','start_event','end_event','name'),encoding='ISO 8859-1') 
    calEvent['start_event'] = pd.to_datetime(calEvent.Date+'_'+calEvent.start_event,format= '%m/%d/%y_%H:%M:%S')
    calEvent['end_event'] = pd.to_datetime(calEvent.Date+'_'+calEvent.end_event,format= '%m/%d/%y_%H:%M:%S')
    calEvent['Date'] = pd.to_datetime(calEvent.Date,format= '%m/%d/%y')

traffic.head()



,Datetime,count
0,2005-04-10 00:00:00,-1
1,2005-04-10 00:05:00,-1
2,2005-04-10 00:10:00,-1
3,2005-04-10 00:15:00,-1
4,2005-04-10 00:20:00,-1


In [3]:
events.dtypes

Date               datetime64[ns]
end_event          datetime64[ns]
start_event        datetime64[ns]
Game attendance             int64
Away team                  object
W/L score                  object
dtype: object

In [ ]:
plt.figure(figsize=(20,10))

start = datetime.datetime.strptime("21-06-2014", "%d-%m-%Y")
end = datetime.datetime.strptime("07-07-2014", "%d-%m-%Y")

plt.plot(traffic[traffic.direction==9].datetime,traffic[traffic.direction==9].num)
plt.plot(traffic[traffic.direction==7].datetime,traffic[traffic.direction==7].num)

plt.show()
print(df.head())


In [ ]:
df["datetime"].dt.normalize().unique()


In [4]:
events.head(100)

,Date,end_event,start_event,Game attendance,Away team,W/L score
0,2005-04-12,2005-04-12 13:10:00,2005-04-12 16:23:00,55892,San Francisco,W 9-8
1,2005-04-13,2005-04-13 19:10:00,2005-04-13 21:48:00,46514,San Francisco,W 4-1
2,2005-04-15,2005-04-15 19:40:00,2005-04-15 21:48:00,51816,San Diego,W 4-0
3,2005-04-16,2005-04-16 19:10:00,2005-04-16 21:52:00,54704,San Diego,W 8-3
4,2005-04-17,2005-04-17 13:10:00,2005-04-17 15:31:00,53402,San Diego,W 6-0
...,...,...,...,...,...,...
76,2005-09-25,2005-09-25 13:10:00,2005-09-25 16:06:00,37846,Pittsburgh,W 9-2
77,2005-09-26,2005-09-26 19:10:00,2005-09-26 22:27:00,36397,Pittsburgh,W 9-4
78,2005-09-27,2005-09-27 19:10:00,2005-09-27 21:33:00,42604,Arizona,L 2-0
79,2005-09-28,2005-09-28 19:10:00,2005-09-28 21:58:00,46424,Arizona,L 4-3


In [5]:
events['Outcome'] = 'W'
events['Score 1'] = 0
events['Score 2'] = 0

events
    
    

,Date,end_event,start_event,Game attendance,Away team,W/L score,Outcome,Score 1,Score 2
0,2005-04-12,2005-04-12 13:10:00,2005-04-12 16:23:00,55892,San Francisco,W 9-8,W,0,0
1,2005-04-13,2005-04-13 19:10:00,2005-04-13 21:48:00,46514,San Francisco,W 4-1,W,0,0
2,2005-04-15,2005-04-15 19:40:00,2005-04-15 21:48:00,51816,San Diego,W 4-0,W,0,0
3,2005-04-16,2005-04-16 19:10:00,2005-04-16 21:52:00,54704,San Diego,W 8-3,W,0,0
4,2005-04-17,2005-04-17 13:10:00,2005-04-17 15:31:00,53402,San Diego,W 6-0,W,0,0
...,...,...,...,...,...,...,...,...,...
76,2005-09-25,2005-09-25 13:10:00,2005-09-25 16:06:00,37846,Pittsburgh,W 9-2,W,0,0
77,2005-09-26,2005-09-26 19:10:00,2005-09-26 22:27:00,36397,Pittsburgh,W 9-4,W,0,0
78,2005-09-27,2005-09-27 19:10:00,2005-09-27 21:33:00,42604,Arizona,L 2-0,W,0,0
79,2005-09-28,2005-09-28 19:10:00,2005-09-28 21:58:00,46424,Arizona,L 4-3,W,0,0


In [7]:
for ii in range(len(events)):
    #temp = re.split('  |- ', str(events.at[ii, 'W/L score']))
    temp1 = str(events.at[ii, 'W/L score']).split(' ')
    temp2 = temp1[1].split('-')
    events.at[ii, 'Outcome'] = temp1[0]
    events.at[ii, 'Score 1'] = int(temp2[0])

    events.at[ii, 'Score 2'] = int(temp2[1][:len(temp2[1])-1])

events

,Date,end_event,start_event,Game attendance,Away team,W/L score,Outcome,Score 1,Score 2
0,2005-04-12,2005-04-12 13:10:00,2005-04-12 16:23:00,55892,San Francisco,W 9-8,W,9,8
1,2005-04-13,2005-04-13 19:10:00,2005-04-13 21:48:00,46514,San Francisco,W 4-1,W,4,1
2,2005-04-15,2005-04-15 19:40:00,2005-04-15 21:48:00,51816,San Diego,W 4-0,W,4,0
3,2005-04-16,2005-04-16 19:10:00,2005-04-16 21:52:00,54704,San Diego,W 8-3,W,8,3
4,2005-04-17,2005-04-17 13:10:00,2005-04-17 15:31:00,53402,San Diego,W 6-0,W,6,0
...,...,...,...,...,...,...,...,...,...
76,2005-09-25,2005-09-25 13:10:00,2005-09-25 16:06:00,37846,Pittsburgh,W 9-2,W,9,2
77,2005-09-26,2005-09-26 19:10:00,2005-09-26 22:27:00,36397,Pittsburgh,W 9-4,W,9,4
78,2005-09-27,2005-09-27 19:10:00,2005-09-27 21:33:00,42604,Arizona,L 2-0,L,2,0
79,2005-09-28,2005-09-28 19:10:00,2005-09-28 21:58:00,46424,Arizona,L 4-3,L,4,3


In [13]:
events['Label 1'] = 0
events['Label 2'] = 0
events['Label 3'] = 0



events.loc[list(events[(events['Outcome'] == 'W') & (events['Score 1']-events['Score 2']  > 3)].index), ['Label 1']] = 1
events.loc[list(events[(events['Outcome'] == 'L') & (events['Score 1']-events['Score 2']  > 3)].index), ['Label 2']] = 1
events.loc[list(events[events['Score 1']-events['Score 2']  <= 3].index), ['Label 3']] = 1


In [14]:
events

,Date,end_event,start_event,Game attendance,Away team,W/L score,Outcome,Score 1,Score 2,Label 1,Label 2,Label 3
0,2005-04-12,2005-04-12 13:10:00,2005-04-12 16:23:00,55892,San Francisco,W 9-8,W,9,8,0,0,1
1,2005-04-13,2005-04-13 19:10:00,2005-04-13 21:48:00,46514,San Francisco,W 4-1,W,4,1,0,0,1
2,2005-04-15,2005-04-15 19:40:00,2005-04-15 21:48:00,51816,San Diego,W 4-0,W,4,0,1,0,0
3,2005-04-16,2005-04-16 19:10:00,2005-04-16 21:52:00,54704,San Diego,W 8-3,W,8,3,1,0,0
4,2005-04-17,2005-04-17 13:10:00,2005-04-17 15:31:00,53402,San Diego,W 6-0,W,6,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
76,2005-09-25,2005-09-25 13:10:00,2005-09-25 16:06:00,37846,Pittsburgh,W 9-2,W,9,2,1,0,0
77,2005-09-26,2005-09-26 19:10:00,2005-09-26 22:27:00,36397,Pittsburgh,W 9-4,W,9,4,1,0,0
78,2005-09-27,2005-09-27 19:10:00,2005-09-27 21:33:00,42604,Arizona,L 2-0,L,2,0,0,0,1
79,2005-09-28,2005-09-28 19:10:00,2005-09-28 21:58:00,46424,Arizona,L 4-3,L,4,3,0,0,1


In [19]:
events['Label 3'].sum()

49